In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.ensemble import GradientBoostingRegressor as gb
from sklearn.ensemble import AdaBoostRegressor as ab
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.metrics import mean_absolute_error as mae
from xgboost import XGBRegressor as xgb

# requesting item information from the API
api_url = "https://api.warframe.market/v1/items"
headers = {"Authorization": "hehe"}
response = requests.get(api_url, headers=headers, timeout=300)

# check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()['payload']['items']
    df = pd.DataFrame(data)

else:
    # error responses
    print(f"Error: {response.status_code} - {response.text}")

# initializing columns for lowest price and predicted lowest price for the next day
df['lowest price available'] = 0
df['predicted lowest price for next day (linear)'] = 0
df['predicted lowest price for next day (rf)'] = 0
df['predicted lowest price for next day (gb)'] = 0
df['predicted lowest price for next day (ab_dt)'] = 0
df['predicted lowest price for next day (xgb)'] = 0  
df['MAE (linear)'] = 0
df['MAE (rf)'] = 0
df['MAE (gb)'] = 0
df['MAE (ab_dt)'] = 0
df['MAE (xgb)'] = 0 

# keyword to filter items
keywords = ['prime']

# loop through each item in the DataFrame
for i in range(len(df['url_name'])):
    item_name = df['url_name'][i]

    # check if item_name contains the keyword
    if all(keyword in item_name for keyword in keywords):

        # requesting specific item information from the API
        api_url = f"https://api.warframe.market/v1/items/{item_name}/orders"
        headers = {"Authorization": "Bearer teehee"}
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            data = response.json()['payload']['orders']
            df1 = pd.DataFrame(data)

            # initialize lists to store user information
            user_ids = []
            statuses = []

            for order in data:
                user_data = order['user']
                user_ids.append(user_data['id'])
                statuses.append(user_data['status'])

            df1['user_id'] = user_ids
            df1['status'] = statuses

        else:
            print(f"Error: {response.status_code} - {response.text}")

        try:
            # filter orders for in-game status
            df1 = df1.loc[(df1['status'] == "ingame")]

            # extracting order information
            df1['t1'] = pd.to_datetime(df1['last_update'])
            df1 = df1.loc[(df1['t1'].dt.year == 2024) & (df1['order_type'] == "sell")]
            mini = df1['platinum'].min()
            df1 = df1.sort_values(by='platinum', ascending=True)

            # filter orders within a certain platinum range
            df1 = df1.loc[(df1['platinum'] < (mini + 20))]
            df['lowest price available'][i] = mini

            # only predicting items with more than 5 orders
            if df1['id'].count() > 5:
                df1['t1'] = pd.to_datetime(df1['last_update'])
                df1['date_numeric'] = df1['t1'].view('int64') // 10**9

                df1['day'] = df1['t1'].dt.day
                day = ['day']
                x = df1[day]
                y = df1['platinum']

                # training and testing sets
                xtrain, xtest, ytrain, ytest = tts(x, y, test_size=0.2, random_state=20)

                # initialize and train the models
                lr_model = lr()
                lr_model.fit(xtrain, ytrain)

                rf_model = rf(n_estimators=100, random_state=20)
                rf_model.fit(xtrain, ytrain)

                gb_model = gb(n_estimators=100, random_state=20)
                gb_model.fit(xtrain, ytrain)

                ada_boost_dt = ab(base_estimator=dtr(max_depth=1), n_estimators=50, random_state=42)
                ada_boost_dt.fit(xtrain, ytrain)

                xgb_model = xgb(n_estimators=100, random_state=20)  
                xgb_model.fit(xtrain, ytrain)  

                # making predictions based on the test set models
                pred_lr = lr_model.predict(xtest)
                pred_rf = rf_model.predict(xtest)
                pred_gb = gb_model.predict(xtest)
                pred_ab_dt = ada_boost_dt.predict(xtest)
                pred_xgb = xgb_model.predict(xtest)  

                # predicting for the next day
                today = dt.datetime.today().day
                next_day = pd.DataFrame({'day': [today + 1]})

                lrpred = lr_model.predict(next_day)
                df['predicted lowest price for next day (linear)'][i] = round(lrpred[0])

                rfpred = rf_model.predict(next_day)
                df['predicted lowest price for next day (rf)'][i] = round(rfpred[0])

                gbpred = gb_model.predict(next_day)
                df['predicted lowest price for next day (gb)'][i] = round(gbpred[0])

                abpred = ada_boost_dt.predict(next_day)
                df['predicted lowest price for next day (ab_dt)'][i] = round(abpred[0])

                xgbpred = xgb_model.predict(next_day) 
                df['predicted lowest price for next day (xgb)'][i] = round(xgbpred[0])

                df['MAE (ab_dt)'] = mae(ytest, pred_ab_dt)
                df['MAE (linear)'][i] =mae(ytest, pred_lr)  
                df['MAE (rf)'][i] = mae(ytest, pred_rf)
                df['MAE (gb)'][i] = mae(ytest, pred_gb)
                df['MAE (xgb)'][i] = mae(ytest, pred_xgb)

            #for no orders are available, we set it as 'NA'
            elif df1['id'].count() == 0:
                df['predicted lowest price for next day (linear)'][i] = 'NA'
                df['predicted lowest price for next day (rf)'][i] = 'NA'
                df['predicted lowest price for next day (gb)'][i] = 'NA'
                df['predicted lowest price for next day (ab_dt)'][i] = 'NA'
                df['predicted lowest price for next day (xgb)'][i] = 'NA'


            #for less than 5 orders, we take the minimum
            else:
                df['predicted lowest price for next day (linear)'][i] = df1['platinum'].min()
                df['predicted lowest price for next day (rf)'][i] = df1['platinum'].min()
                df['predicted lowest price for next day (gb)'][i] = df1['platinum'].min()
                df['predicted lowest price for next day (ab_dt)'][i] = df1['platinum'].min()
                df['predicted lowest price for next day (xgb)'][i] = df1['platinum'].min()
            del df1
            
        except NameError:
            print("API request failed")

    #drop rows that do not meet the keyword criteria
    else:
        df.drop(i, inplace=True)

#save the dataframe into an excel file
excel_file_path = "warframe_market_data.xlsx"
df.to_excel(excel_file_path, index=False)